In [9]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json

import time

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   

# Load data

From s00_generate_network.ipynb, import the generated network and data files.

In [2]:
DATASET = Path(r"data\grid_8x8\v1\data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)

# all edges ON (example); add node/edge 0s as needed
states = {eid: 1 for eid in edges.keys()}

k_val, state, _ = fbg.eval_global_conn_k(states, G_base)
print("k =", k_val, "state =", state)

k = 2 state = 2


In [3]:
s_fun = lambda comps_st: fbg.eval_global_conn_k(comps_st, G_base)
row_names = list(edges.keys())
n_state = 2  # binary states: 0, 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

From s01_find_rules.ipynb, load rules for system state 1 and 2.

In [4]:
TSUMPATH = Path("tsum_res") 
rules_dict_mat_surv = {}
rules_dict_mat_fail = {}

for sys_surv_st in [1, 2]:  # either 1 or 2
    rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_st}.pt", map_location="cpu")
    rules_mat_surv = rules_mat_surv.to(device)
    rules_dict_mat_surv[sys_surv_st] = rules_mat_surv
    rules_mat_fail = torch.load(TSUMPATH / f"rules_leq_{sys_surv_st-1}.pt", map_location="cpu")
    rules_mat_fail = rules_mat_fail.to(device)
    rules_dict_mat_fail[sys_surv_st] = rules_mat_fail

# Calculate probabilities

## Marginal probability

In [5]:
cond_probs = tsum.get_comp_cond_sys_prob_multi(
                rules_dict_mat_surv,
                rules_dict_mat_fail,
                probs,
                comps_st_cond = {},
                row_names = row_names,
                s_fun = s_fun
            )
print("P(sys):", cond_probs)

P(sys): {0: 0.000413, 1: 0.082559, 2: 0.917028}


## Conditional probability given "one" component's survival

In [ ]:
for x in row_names[:2]:  # first two components
    st = time.time()
    pr_cond = tsum.get_comp_cond_sys_prob_multi(
        rules_dict_mat_surv,
        rules_dict_mat_fail,
        probs,
        comps_st_cond = {x: 1},
        row_names = row_names,
        s_fun = s_fun
    )
    print(f"P(sys | {x}=1):", pr_cond, f"time: {time.time() - st:.1f} sec")

P(sys | e0=1): {0: 0.00032, 1: 0.07313, 2: 0.92655} time: 32.5 sec
P(sys | e1=1): {0: 0.000324, 1: 0.073324, 2: 0.926352} time: 34.2 sec


## Conditional probability given "two" component's survival

In [ ]:
for i, x in enumerate(row_names[:3]):  # first a few components for test
    for j in row_names[i+1:3]:
        st = time.time()
        pr_cond = tsum.get_comp_cond_sys_prob_multi(
            rules_dict_mat_surv,
            rules_dict_mat_fail,
            probs,
            comps_st_cond = {x: 1, j: 1},
            row_names = row_names,
            s_fun = s_fun
        )
        print(f"P(sys | {x}=1, {j}=1):", pr_cond, f"time: {time.time() - st:.1f} sec")


P(sys | e0=1, e1=1): {0: 0.000312, 1: 0.064028, 2: 0.93566} time: 33.0 sec
P(sys | e0=1, e2=1): {0: 0.000327, 1: 0.072467, 2: 0.927206} time: 30.5 sec
P(sys | e1=1, e2=1): {0: 0.00033, 1: 0.072528, 2: 0.927142} time: 31.5 sec
